In [1]:
!pip install tpot --quiet
!pip install imblearn --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pdpbox 0.2.1 requires matplotlib==3.1.1, but you have matplotlib 3.4.3 which is incompatible.
hypertools 0.7.0 requires scikit-learn!=0.22,<0.24,>=0.19.1, but you have scikit-learn 1.0 which is incompatible.


cambio naive. Probando integración

In [2]:
import pandas as pd
import numpy as np
import warnings
from tpot import TPOTClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedShuffleSplit, RepeatedStratifiedKFold

warnings.filterwarnings('ignore')

In [3]:
artists_n_genres = pd.read_csv('../input/artistspopulargenre/artists_n_genres.csv')
artists_n_genres.head()

,Unnamed: 0,genres,artists,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,count,popular_genre
0,0,['show tunes'],"""Cats"" 1981 Original London Cast",0.590111,0.467222,250318.555556,0.394003,0.011400,0.290833,-14.448000,0.210389,117.518111,0.389500,38.333333,5,1,9,show tunes
1,8,"['comedy rock', 'comic', 'parody']","""Weird Al"" Yankovic",0.173145,0.662787,218948.196721,0.695393,0.000050,0.161102,-9.768705,0.084536,133.031180,0.751344,34.229508,9,1,122,comic
2,9,"['emo rap', 'florida rap', 'sad rap', 'undergr...",$NOT,0.544467,0.789800,137910.466667,0.532933,0.023063,0.180300,-9.149267,0.293687,112.344800,0.480700,67.533333,1,1,15,underground hip hop
3,10,"['dark trap', 'meme rap']",$atori Zoom,0.239000,0.883000,141519.000000,0.625000,0.000000,0.076500,-4.098000,0.245000,126.677000,0.871000,67.000000,6,1,2,dark trap
4,12,"['asian american hip hop', 'cali rap', 'west c...",$tupid Young,0.148100,0.854000,190572.000000,0.683000,0.000002,0.188500,-6.997000,0.221000,100.724500,0.625500,57.500000,1,1,2,cali rap


In [4]:
artists_n_genres.drop(['genres', 'Unnamed: 0'], axis=1, inplace=True)
artists_n_genres.head()

,artists,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,count,popular_genre
0,"""Cats"" 1981 Original London Cast",0.590111,0.467222,250318.555556,0.394003,0.011400,0.290833,-14.448000,0.210389,117.518111,0.389500,38.333333,5,1,9,show tunes
1,"""Weird Al"" Yankovic",0.173145,0.662787,218948.196721,0.695393,0.000050,0.161102,-9.768705,0.084536,133.031180,0.751344,34.229508,9,1,122,comic
2,$NOT,0.544467,0.789800,137910.466667,0.532933,0.023063,0.180300,-9.149267,0.293687,112.344800,0.480700,67.533333,1,1,15,underground hip hop
3,$atori Zoom,0.239000,0.883000,141519.000000,0.625000,0.000000,0.076500,-4.098000,0.245000,126.677000,0.871000,67.000000,6,1,2,dark trap
4,$tupid Young,0.148100,0.854000,190572.000000,0.683000,0.000002,0.188500,-6.997000,0.221000,100.724500,0.625500,57.500000,1,1,2,cali rap


In [5]:
artists_n_genres = artists_n_genres[artists_n_genres['popular_genre'].map(artists_n_genres['popular_genre'].value_counts()) > 100]

In [6]:
artists_n_genres['popular_genre_code'] = artists_n_genres['popular_genre'].astype('category').cat.codes
artists_n_genres.head()

,artists,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,count,popular_genre,popular_genre_code
0,"""Cats"" 1981 Original London Cast",0.590111,0.467222,250318.555556,0.394003,0.011400,0.290833,-14.448000,0.210389,117.518111,0.389500,38.333333,5,1,9,show tunes,32
2,$NOT,0.544467,0.789800,137910.466667,0.532933,0.023063,0.180300,-9.149267,0.293687,112.344800,0.480700,67.533333,1,1,15,underground hip hop,34
5,$uicideBoy$,0.141485,0.749344,146386.392000,0.635552,0.045675,0.202253,-6.631304,0.156108,115.022024,0.287286,61.800000,1,1,125,underground hip hop,34
6,'In The Heights' Original Broadway Company,0.624769,0.563615,314023.615385,0.457692,0.000009,0.204385,-8.338462,0.152454,117.006846,0.467538,47.692308,7,1,13,show tunes,32
7,'Legally Blonde' Ensemble,0.553889,0.648444,304211.888889,0.441111,0.000025,0.214667,-11.459778,0.495111,114.808444,0.524778,48.666667,2,1,9,show tunes,32


In [7]:
stratified_split = StratifiedShuffleSplit(
    test_size=.3,
    random_state=333,
    n_splits=2
)

X_train, X_test, y_train, y_test = ([] for i in range(4))

X = artists_n_genres.drop(['popular_genre_code', 'artists', 'popular_genre'], axis=1)
y = artists_n_genres['popular_genre_code']


for train_idx, test_idx in stratified_split.split(np.zeros(len(X)), y):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

In [8]:
y_train

9221     19
9449      6
10516     4
14416    31
2066     30
         ..
18784     3
14279    14
10009    30
13511     2
17804     4
Name: popular_genre_code, Length: 5182, dtype: int8

In [9]:
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_train, y_train)

In [10]:
generations=3
population_size=5
cv = RepeatedStratifiedKFold(n_splits=2, random_state=1)

pipeline_optimizer = TPOTClassifier(
    verbosity=2, 
    random_state=42,
    memory='auto',
    n_jobs=-1,
    scoring='f1_weighted',
    cv=cv,
    template='Transformer-Selector-Classifier',
    config_dict='TPOT light',
)

In [11]:
pipeline_optimizer.fit(X_train, y_train)

Optimization Progress:   0%|          | 0/10100 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.3793538094981377

Generation 2 - Current best internal CV score: 0.3802041491357712

Generation 3 - Current best internal CV score: 0.38111538053745286

Generation 4 - Current best internal CV score: 0.38111538053745286

Generation 5 - Current best internal CV score: 0.38308981799687125

Generation 6 - Current best internal CV score: 0.38668884495577444

Generation 7 - Current best internal CV score: 0.38668884495577444

Generation 8 - Current best internal CV score: 0.38668884495577444

Generation 9 - Current best internal CV score: 0.38668884495577444

Generation 10 - Current best internal CV score: 0.38668884495577444

Generation 11 - Current best internal CV score: 0.38668884495577444

Generation 12 - Current best internal CV score: 0.38668884495577444

Generation 13 - Current best internal CV score: 0.38668884495577444

Generation 14 - Current best internal CV score: 0.38668884495577444

Generation 15 - Current best internal CV sco

TPOTClassifier(config_dict='TPOT light',
               cv=RepeatedStratifiedKFold(n_repeats=10, n_splits=2, random_state=1),
               memory='auto', n_jobs=-1, random_state=42, scoring='f1_weighted',
               template='Transformer-Selector-Classifier', verbosity=2)